In [0]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
#from google.colab.patches import cv2_imshow

class WorkspaceMap:
    def __init__(self,radius,clearance):
        
        self.radius = radius
        self.clearance = clearance
        self.height = 200
        self.width = 300
        #self.Surface = pg.display.set_mode((self.width,self.height))
        
        self.padding = self.radius+self.clearance
        padding = self.padding
        height = self.height
        width = self.width

        self.coord_poly = np.array([(25-padding, height-(185+padding)),
                               (75+padding, height-(185+padding)),
                               (100+padding, height-(150+padding)),
                               (75+padding, height-(120-padding)),
                               (50+padding, height-(150-padding)),
                               (20-padding, height-(120-padding))], dtype=np.int32)
        
        self.coord_rect = np.array([(30-padding, height-(67.5+padding)),
                      (35+padding, height-(76+padding)),
                      (100+padding, height-(38.6-padding)),
                      (95-padding, height-(30-padding))], dtype = np.int32).reshape((-1,1,2))
        
        self.coord_rhom = np.array([(225, height-(40+padding)),
                      (250+ padding, height-25),
                      (225, height-(10-padding)),
                      (200-padding, height-25)], dtype=np.int32).reshape((-1,1,2))
        self.circle = [(25+padding),(225,50)]
        self.ellipse = [(80+padding,40+padding),(150,100)]


    def checkPoly(Poly,x,y):
        Poly_x = Poly[:,0]
        Poly_y = Poly[:,1]

        m12 = (Poly_y[0]-Poly_y[1])/(Poly_x[0]-Poly_x[1])
        m23 = (Poly_y[1]-Poly_y[2])/(Poly_x[1]-Poly_x[2])
        m34 = (Poly_y[2]-Poly_y[3])/(Poly_x[2]-Poly_x[3])
        m45 = (Poly_y[3]-Poly_y[4])/(Poly_x[3]-Poly_x[4])
        m56 = (Poly_y[4]-Poly_y[5])/(Poly_x[4]-Poly_x[5])
        m16 = (Poly_y[0]-Poly_y[5])/(Poly_x[0]-Poly_x[5])
        m25 = (Poly_y[1]-Poly_y[4])/(Poly_x[1]-Poly_x[4])

        if (y >= (x-Poly_x[0])*m12+Poly_y[0] and y >= (x-Poly_x[0])*m16+Poly_y[0] and y <= (x-Poly_x[4])*m56 + Poly_y[4] and y <= (x-Poly_x[1])*m25+Poly_y[1]):
          return True
        
        elif (y >= (x-Poly_x[1])*m25+Poly_y[1] and y <= (x-Poly_x[3])*m45+Poly_y[3] and y <= (x-Poly_x[2])*m34+Poly_y[2] and y >= (x-Poly_x[1])*m23+Poly_y[1]):
          return True

        return False

    def checkRect(rect,x,y):
        rect_x = rect[:,0]
        rect_y = rect[:,1]

        m12 = (rect_y[0]-rect_y[1])/(rect_x[0]-rect_x[1])
        m23 = (rect_y[1]-rect_y[2])/(rect_x[1]-rect_x[2])
        m34 = (rect_y[2]-rect_y[3])/(rect_x[2]-rect_x[3])
        m14 = (rect_y[0]-rect_y[3])/(rect_x[0]-rect_x[3])

        if (y>=(x-rect_x[0])*m12+rect_y[0] and y>=(x-rect_x[1])*m23+rect_y[1] and y<=(x-rect_x[2])*m34+rect_y[2] and y<=(x-rect_x[0])*m14+rect_y[0]):
          return True
        
        return False

    def checkRhom(rhom,x,y):
        rhom_x = rhom[:,0]
        rhom_y = rhom[:,1]

        m12 = (rhom_y[0]-rhom_y[1])/(rhom_x[0]-rhom_x[1])
        m23 = (rhom_y[1]-rhom_y[2])/(rhom_x[1]-rhom_x[2])
        m34 = (rhom_y[2]-rhom_y[3])/(rhom_x[2]-rhom_x[3])
        m14 = (rhom_y[0]-rhom_y[3])/(rhom_x[0]-rhom_x[3])

        if (y>=(x-rhom_x[0])*m12+rhom_y[0] and y<=(x-rhom_x[1])*m23+rhom_y[1] and y<=(x-rhom_x[2])*m34+rhom_y[2] and y>=(x-rhom_x[0])*m14+rhom_y[0]):
          return True
        
        return False

    def checkCirc(circle,x,y):
        a = circle[1][0]
        b = circle[1][1]
        r = circle[0]

        if ((x-a)**2 + (y-b)**2 <= r**2):
          return True
        return False

    def checkEllipse(ellipse,x,y):
        A = ell[0][0]/2
        B = ell[0][1]/2

        a = ell[1][0]
        b = ell[1][1]

        if (((x-a)/A)**2 + ((y-b)/B)**2 <= 1):
          return True
        return False

    def checkObstacle(self,x,y):
      Poly = self.coord_poly
      rect = self.coord_rect
      rhom = self.coord_rhom
      circle = self.circle
      ellipse = self.ellipse

      if checkPoly(Poly,x,y) or checkRect(rect,x,y) or checkRhom(rhom,x,y) or checkCirc(circle,x,y) or checkEllipse(ellipse,x,y):
        return True

    #def start_visualization(self,img,initial_pos,visited_nodes,path):
    def getMap(self, coord_Poly,coord_rect,coord_rhom):
        height = self.height
        width = self.width
        padding =  self.padding
        BLUE_COLOR = (255,0,0)
        img = np.zeros((self.height,self.width,3),dtype=np.uint8)

        if  padding:
            cv2.rectangle(img, (0,0), (width-1, height-1), BLUE_COLOR, padding)
        
        cv2.fillPoly(img, [coord_poly], BLUE_COLOR)
        cv2.fillConvexPoly(img, coord_rect, BLUE_COLOR)
        cv2.fillConvexPoly(img, coord_rhom, BLUE_COLOR)
        cv2.circle(img, (225, height-150), (25+padding), BLUE_COLOR, -1)
        cv2.ellipse(img, (150, height-100), (40+padding, 20+padding), 0, 0, 360, BLUE_COLOR, -1)
     
        return img

In [64]:
Map = WorkspaceMap(0,0)
Out = Map.checkObstacle(150,100)
print(Out)


# def checkPoly(Poly,x,y):
#   Poly_x = Poly[:,0]
#   Poly_y = Poly[:,1]

#   m12 = (Poly_y[0]-Poly_y[1])/(Poly_x[0]-Poly_x[1])
#   m23 = (Poly_y[1]-Poly_y[2])/(Poly_x[1]-Poly_x[2])
#   m34 = (Poly_y[2]-Poly_y[3])/(Poly_x[2]-Poly_x[3])
#   m45 = (Poly_y[3]-Poly_y[4])/(Poly_x[3]-Poly_x[4])
#   m56 = (Poly_y[4]-Poly_y[5])/(Poly_x[4]-Poly_x[5])
#   m16 = (Poly_y[0]-Poly_y[5])/(Poly_x[0]-Poly_x[5])
#   m25 = (Poly_y[1]-Poly_y[4])/(Poly_x[1]-Poly_x[4])

#   if (y >= (x-Poly_x[0])*m12+Poly_y[0] and y >= (x-Poly_x[0])*m16+Poly_y[0] and y <= (x-Poly_x[4])*m56 + Poly_y[4] and y <= (x-Poly_x[1])*m25+Poly_y[1]):
#     return True
  
#   elif (y >= (x-Poly_x[1])*m25+Poly_y[1] and y <= (x-Poly_x[3])*m45+Poly_y[3] and y <= (x-Poly_x[2])*m34+Poly_y[2] and y >= (x-Poly_x[1])*m23+Poly_y[1]):
#     return True

#   return False

# def checkRect(rect,x,y):
#   rect_x = rect[:,0]
#   rect_y = rect[:,1]

#   m12 = (rect_y[0]-rect_y[1])/(rect_x[0]-rect_x[1])
#   m23 = (rect_y[1]-rect_y[2])/(rect_x[1]-rect_x[2])
#   m34 = (rect_y[2]-rect_y[3])/(rect_x[2]-rect_x[3])
#   m14 = (rect_y[0]-rect_y[3])/(rect_x[0]-rect_x[3])

#   if (y>=(x-rect_x[0])*m12+rect_y[0] and y>=(x-rect_x[1])*m23+rect_y[1] and y<=(x-rect_x[2])*m34+rect_y[2] and y<=(x-rect_x[0])*m14+rect_y[0]):
#     return True
  
#   return False

# def checkRhom(rhom,x,y):
#   rhom_x = rhom[:,0]
#   rhom_y = rhom[:,1]

#   m12 = (rhom_y[0]-rhom_y[1])/(rhom_x[0]-rhom_x[1])
#   m23 = (rhom_y[1]-rhom_y[2])/(rhom_x[1]-rhom_x[2])
#   m34 = (rhom_y[2]-rhom_y[3])/(rhom_x[2]-rhom_x[3])
#   m14 = (rhom_y[0]-rhom_y[3])/(rhom_x[0]-rhom_x[3])

#   if (y>=(x-rhom_x[0])*m12+rhom_y[0] and y<=(x-rhom_x[1])*m23+rhom_y[1] and y<=(x-rhom_x[2])*m34+rhom_y[2] and y>=(x-rhom_x[0])*m14+rhom_y[0]):
#     return True
  
#   return False

# def checkCirc(circle,x,y):
#   a = circle[1][0]
#   b = circle[1][1]
#   r = circle[0]

#   if ((x-a)**2 + (y-b)**2 <= r**2):
#     return True
#   return False

# def checkEllipse(ell,x,y):
#   A = ell[0][0]/2
#   B = ell[0][1]/2

#   a = ell[1][0]
#   b = ell[1][1]

#   if (((x-a)/A)**2 + ((y-b)/B)**2 <= 1):
#     return True
#   return False



Pout = checkPoly(Poly,75,50)
Cout = checkCirc(Circ,225,50)
Elout = checkEllipse(ell,130,100)
print("Poly out",Pout)
print("Circle out",Cout)
print("Ellipse out",Elout)

# img = Map.getMap()
# cv2_imshow(img)

IndexError: ignored